In [1]:
# 단 웹드라이버 위치 수정 필요
# 변수 추출# 드라이버 위치 설정
driver_loc = "d:/chromedriver/chromedriver.exe"

targetspec=['센서', '측정', '체크', '보행수', '심전도']

# 라이브러리 선언
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd


# 드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")

 


# 웹 드라이버 정의
driver = webdriver.Chrome(driver_loc, options=options)

 

dna_url = "http://prod.danawa.com/list/?cate=12215657"
driver.get(dna_url)
time.sleep(1)

 

path1='//*[@id="searchMakerRep702"]' #삼성전자
path2='//*[@id="searchMakerRep79466"]' #샤오미
path3='//*[@id="searchMakerRep74710"]' #화웨이
path4='//*[@id="searchMakerRep20779338"]' #SM코퍼레이션
path5='//*[@id="productListArea"]/div[2]/div[2]/div[2]/select'
path6='//*[@id="productListArea"]/div[2]/div[2]/div[2]/select/option[2]'

 

driver.find_element_by_xpath(path1).click()
driver.find_element_by_xpath(path2).click()
driver.find_element_by_xpath(path3).click()
driver.find_element_by_xpath(path4).click()
time.sleep(2)

 

driver.find_element_by_xpath(path5).click()
driver.find_element_by_xpath(path6).click()
time.sleep(2)

 

html = driver.page_source

 

import bs4
bs = bs4.BeautifulSoup(html, "html.parser")

 

prod_info = bs.findAll("div", {"class":"prod_main_info"})
del prod_info[0] #0번째의 추천상품은 중복되므로 제거


nameList = [] #상품이름을 담을 리스트
review_num_List = [] #평점갯수를 담을 리스트
pointList = [] #평점을 담을 리스트
linkList = []

for i in range(0, len(prod_info)):
    try :
        pointList.append(prod_info[i].find("div", {"class":"point_num"}).text)
        review_num_List.append(prod_info[i].find("a", {"onmousedown":"_trkEventLog('15상품리스트_상품의견')"}).find("strong").text+"건")
        nameList.append(prod_info[i].find("p", {"class":"prod_name"}).find("a").text.replace("\t","").replace("\n",""))
        linkList.append(prod_info[i].find("p", {"class":"prod_name"}).find("a").attrs["href"])
    except :
        pointList.append('평점없음') #평점이 없는경우 "평점없음"을 리스트에 담기
        review_num_List.append('리뷰없음') #리뷰가 없는경우 "리뷰없음"을 리스트에 담기
        nameList.append(prod_info[i].find("p", {"class":"prod_name"}).find("a").text.replace("\t","").replace("\n","")) #replace를 할때 try에서 실패할경우 except로 넘어오므로 한번더 except에서 리스트에 담기
        linkList.append(prod_info[i].find("p", {"class":"prod_name"}).find("a").attrs["href"])
 

priceList = []
temppriceList = []
for i in range(0, len(prod_info)):
    for j in prod_info[i].findAll("li"):
        temppriceList.append(j.find("p", {"class":"price_sect"}).find("strong").text) #상품에대한 모든 가격정보를 temppriceList에 추가
    temppriceList.sort() #sort이용 오름차순 정리
    priceList.append(temppriceList[0]) #최소가격만 priceList에 추가
    temppriceList.clear()

 
tempt = []
splitList = []
conList = []
for k in prod_info:
    try :#splitList에 prod_info의 요소를 하나씩 담기
        splitList = k.find("div", {"class":"spec_list"}).text.split("/")
    except :
        splitList = k.find("li", {"class":"spec_item"}).text.split("/")
    for i in range(0, len(splitList)):
        for j in targetspec:#원하는 타겟스펙이 있는 항목은 tempt리스트에 담기
            if j in splitList[i]:
                tempt.append(splitList[i]) #
            else : 
                continue
    conList.append((" ".join(tempt)).replace("\t","")) #tempt에 담긴 리스트를 문자열로 변환해서 conList에 담기
    tempt = []
    splitList = []

 

pd.DataFrame(zip(nameList, conList, pointList, review_num_List, priceList, linkList), columns = ["상품명", "스펙", "평점", "상품평 건수","가격", "링크"])


,상품명,스펙,평점,상품평 건수,가격,링크
0,삼성전자 갤럭시 워치 액티브2 40mm 알루미늄 케이스,심박센서 가속도센서 만보기센서 자이로센서 기압센서 보행수 심...,97점,459건,"236,290",http://prod.danawa.com/info/?pcode=8964230&cat...
1,삼성전자 갤럭시 워치 액티브2 44mm 스테인리스 스틸 케이스,심박센서 가속도센서 만보기센서 자이로센서 기압센서 보행수 심...,97점,321건,"335,710",http://prod.danawa.com/info/?pcode=9201189&cat...
2,삼성전자 갤럭시 워치 액티브,심박센서 가속도센서 자이로센서 기압센서 모션센서 보행수 심박...,96점,854건,"164,570",http://prod.danawa.com/info/?pcode=7299064&cat...
3,삼성전자 갤럭시 워치 액티브2 44mm 알루미늄 케이스,심박센서 가속도센서 만보기센서 자이로센서 기압센서 보행수 심...,96점,231건,"274,460",http://prod.danawa.com/info/?pcode=9200895&cat...
4,샤오미 화미 어메이즈핏 GTS,심박센서 가속도센서 기압센서 나침반센서 조도센서,96점,225건,"120,700",http://prod.danawa.com/info/?pcode=10360533&ca...
5,샤오미 헬로우 스마트워치,심박센서 가속도센서 심박측정,55점,104건,"25,920",http://prod.danawa.com/info/?pcode=10319637&ca...
6,삼성전자 갤럭시 워치 46mm,심박센서 가속도센서 자이로센서 기압센서 조도센서 보행수 심박측정,95점,"1,032건","294,820",http://prod.danawa.com/info/?pcode=6378431&cat...
7,샤오미 화미 어메이즈핏 GTR 47mm,심박센서 가속도센서 기압센서 나침반센서 조도센서 보행수 심박측정,94점,99건,"134,000",http://prod.danawa.com/info/?pcode=10240971&ca...
8,삼성전자 갤럭시 워치 액티브2 40mm 스테인리스 스틸 케이스,심박센서 가속도센서 만보기센서 자이로센서 기압센서 보행수 심...,94점,119건,"310,190",http://prod.danawa.com/info/?pcode=9201093&cat...
9,삼성전자 갤럭시 워치 액티브2 44mm 골프 에디션,심박센서 가속도센서 만보기센서 자이로센서 기압센서 보행수 심...,99점,117건,"399,290",http://prod.danawa.com/info/?pcode=10596255&ca...
